# Prepare Census Block OD Data

In [ ]:
import geopandas as gpd
import pandas as pd
from pathlib import Path
import json
import requests
import pickle

from bikewaysim.paths import config
from bikewaysim.framework import bikewaysim_functions

# Retrieve LODES7 data

In [ ]:
state_two_letter = "ga"
YYYY = 2010
lodes_url_base = f"https://lehd.ces.census.gov/data/lodes/LODES7/{state_two_letter}/od/{state_two_letter}_od_main_JT00_{YYYY}.csv.gz"
lodes = pd.read_csv(lodes_url_base)

#processs df
rename_dict = {
    'w_geocode':'work_blockid',
    'h_geocode':'home_blockid',
    'S000': 'total_jobs'  
}
lodes.rename(columns=rename_dict, inplace=True)
lodes = lodes[['work_blockid','home_blockid','total_jobs']]


# Census Blocks

In [ ]:
#study area for masking blocks
studyarea = gpd.read_file(config['studyarea_fp'])#.to_crs(config['projected_crs_epsg'])
studyarea = studyarea.unary_union.envelope

In [ ]:
#bring in census blocks (get from https://www.census.gov/cgi-bin/geo/shapefiles/index.php)
#TODO script downloading the blocks
blocks = gpd.read_file("/Users/tannerpassmore/Documents/BikewaySim/Savannah/OD Creation/tl_2010_13_tabblock10.shp",mask=studyarea).to_crs(config['projected_crs_epsg'])
blocks.rename(columns={'GEOID10':'blockid'},inplace=True)
blocks = blocks[['blockid','geometry']]

# Data Wrangling

In [ ]:
#make sure type are the same
lodes['work_blockid'] = lodes['work_blockid'].astype(int)
lodes['home_blockid'] = lodes['home_blockid'].astype(int)
blocks['blockid'] = blocks['blockid'].astype(int)

In [ ]:
#filter lodes data
print('lodes:',lodes.shape[0],'blocks:',blocks.shape[0])
lodes = lodes[lodes['work_blockid'].isin(blocks['blockid']) & lodes['home_blockid'].isin(blocks['blockid'])]
blocks = blocks[blocks['blockid'].isin(lodes['work_blockid']) | blocks['blockid'].isin(lodes['home_blockid'])]
print('lodes:',lodes.shape[0],'blocks:',blocks.shape[0])

Only 3 mile trips

In [ ]:
block_geo = dict(zip(blocks['blockid'],blocks['geometry'].centroid))
home = gpd.GeoSeries(lodes['home_blockid'].map(block_geo),crs=blocks.crs)
work = gpd.GeoSeries(lodes['work_blockid'].map(block_geo),crs=blocks.crs)
lodes = lodes[home.distance(work) <= 3 * 5280]

In [ ]:
#rename to make it easier to work with teh code for tazs
#assume home is the origin
lodes.columns = ['dest_taz','orig_taz','trips']
blocks.columns = ['tazid','geometry']

# Snap to network

In [ ]:
# import network nodes
nodes = gpd.read_file(config['network_fp']/"final_network.gpkg",layer="nodes")

with (config['calibration_fp']/"calibration_network.pkl").open('rb') as fh:
    links,turns = pickle.load(fh)
del turns

nodes0 = links['A'].append(links['B'])
nodes = nodes[nodes['N'].isin(nodes0)]

In [ ]:
blocks = bikewaysim_functions.snap_ods_to_network(blocks,nodes)

#knock out if over 1000 feet
blocks = blocks[blocks['dist']<=500]

In [ ]:
snapped_ods = dict(zip(blocks['tazid'],blocks['N']))
lodes['orig_N'] = lodes['orig_taz'].map(snapped_ods)
lodes['dest_N'] = lodes['dest_taz'].map(snapped_ods)

In [ ]:
lodes.dropna(inplace=True)

In [ ]:
trip_origins_per_taz = lodes.groupby('orig_taz')['trips'].sum().reset_index()
trip_origins_per_taz.columns = ['tazid','trip_origins']

In [ ]:
# no trips
no_trips = blocks[blocks['tazid'].isin(trip_origins_per_taz['tazid'])==False]
no_trips

In [ ]:
blocks = blocks.merge(trip_origins_per_taz,on='tazid')

In [ ]:
blocks.to_file(config['bikewaysim_fp']/'ods.gpkg',layer='trip_origins')

In [ ]:
# trip origins per TAZ
#export the taz areas
blocks.to_crs(config['projected_crs_epsg']).to_file(config['bikewaysim_fp']/'map_layers.gpkg',layer='taz_polygons')
#export the taz centroids
taz_centroids = blocks.copy().to_crs(config['projected_crs_epsg'])
taz_centroids.geometry = taz_centroids.geometry.centroid
taz_centroids.to_crs(config['projected_crs_epsg']).to_file(config['bikewaysim_fp']/'map_layers.gpkg',layer='taz_centroids')

In [ ]:
studyarea = gpd.read_file(config['studyarea_fp'])
blocks0 = gpd.read_file("/Users/tannerpassmore/Documents/BikewaySim/Savannah/OD Creation/tl_2010_13_tabblock10.shp",mask=studyarea).to_crs(config['projected_crs_epsg'])
blocks0.rename(columns={'GEOID10':'tazid'},inplace=True)
blocks0['tazid'] = blocks0['tazid'].astype(int)
blocks0 = blocks0[['tazid','geometry']]
blocks0 = blocks0[blocks0['tazid'].isin(blocks['tazid'])==False]
blocks0.plot()

In [ ]:
blocks0.to_file(config['bikewaysim_fp']/'ods.gpkg',layer='filtered_out_or_zero')

# Export

In [ ]:
print(lodes['trips'].sum(),'trips across',lodes.shape[0],'unique OD pairs')#,tripData['hh_id'].nunique(),'persons')
print('and',blocks.shape[0],'census blocks')

In [ ]:
# #export the taz areas
# blocks.to_file(config['bikewaysim_fp']/'map_layers.gpkg',layer='block_polygons')

# #export the taz centroids
# blocks_centroids = blocks.copy()
# blocks_centroids.geometry = blocks_centroids.geometry.centroid
# blocks_centroids.to_crs(config['projected_crs_epsg']).to_file(config['bikewaysim_fp']/'map_layers.gpkg',layer='block_centroids')

#export the ods
lodes.to_csv(config['bikewaysim_fp']/'od_matrix.csv',index=False)

Exploring past here

In [ ]:
lodes

In [ ]:
# works = lodes[['work_blockid','total_jobs']].copy()
# works = works.groupby('work_blockid')['total_jobs'].sum().reset_index()
# works = pd.merge(works,blocks[['blockid','geometry']],left_on="work_blockid",right_on='blockid')
# works = gpd.GeoDataFrame(works,geometry='geometry')
# works.explore('total_jobs')
# merged = pd.merge(lodes,blocks[['blockid','geometry']],left_on="work_blockid",right_on='blockid',how='left')
# merged.drop(columns=['blockid'],inplace=True)
# merged.rename(columns={'geometry':'work_geo'},inplace=True)
# merged = pd.merge(merged,blocks[['blockid','geometry']],left_on="home_blockid",right_on='blockid',how='left')
# merged.drop(columns=['blockid'],inplace=True)
# merged.rename(columns={'geometry':'home_geo'},inplace=True)
# merged = merged[merged['home_geo'].notna() & merged['work_geo'].notna()]
# random_id = merged['work_blockid'].sample(1).item()
# subset = merged.loc[merged['work_blockid']==random_id].copy()
# subset = gpd.GeoDataFrame(subset,geometry='home_geo')
# m = subset.explore()
# blocks.loc[blocks['blockid']==random_id,'geometry'].explore()
# work = blocks.loc[blocks['blockid']==random_id,'geometry'].to_crs('epsg:4326').item().centroid
# x, y = list(work.coords)[0]
# import folium
# folium.Circle((y,x),radius=100,tooltip='WORK',kwargs={'color':'green'}).add_to(m)
# m
# merged['work_geo'] = gpd.GeoSeries(merged['work_geo']).centroid
# merged['home_geo'] = gpd.GeoSeries(merged['home_geo']).centroid
# # import geopandas as gpd
# # import pandas as pd
# # from pathlib import Path
# # import json

# # config = json.load((Path.cwd().parent / 'config.json').open('rb'))
# # network_fp = Path(config['project_directory']) / "Network"
# # osmdwnld_fp = Path(config['project_directory']) / "OSM_Download"
# # elevation_fp = Path(config['project_directory']) / "Elevation"
# # cyclinginfra_fp = Path(config['project_directory']) / "Cycling_Infra_Dating"
# # calibration_fp = Path(config['project_directory']) / "Calibration"



# #'foot' in blocks.crs.axis_info[0].unit_name

# #df.crs.axis_info[0].unit_name